In [ ]:
from tensorflow.keras.layers import Input, Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Reshape
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam

# Define the generator
def build_generator(latent_dim, time_steps, features):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=latent_dim))
    model.add(RepeatVector(time_steps))
    model.add(LSTM(128, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(features, activation='sigmoid')))
    return model

# Define the discriminator
def build_discriminator(time_steps, features):
    model = Sequential()
    model.add(LSTM(128, input_shape=(time_steps, features), return_sequences=True))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Build and compile the GAN
latent_dim = 100
time_steps = train_sequences.shape[1]
features = train_sequences.shape[2]

generator = build_generator(latent_dim, time_steps, features)
discriminator = build_discriminator(time_steps, features)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

discriminator.trainable = False
gan_input = Input(shape=(latent_dim,))
generated_sequence = generator(gan_input)
gan_output = discriminator(generated_sequence)
gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

# Training the GAN
def train_gan(gan, generator, discriminator, epochs, batch_size, train_sequences, latent_dim):
    for epoch in range(epochs):
        # Train discriminator
        idx = np.random.randint(0, train_sequences.shape[0], batch_size)
        real_sequences = train_sequences[idx]
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        generated_sequences = generator.predict(noise)
        
        d_loss_real = discriminator.train_on_batch(real_sequences, np.ones((batch_size, 1)))
        d_loss_fake = discriminator.train_on_batch(generated_sequences, np.zeros((batch_size, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
        
        # Train generator
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_y = np.array([1] * batch_size)
        g_loss = gan.train_on_batch(noise, valid_y)
        
        print(f"{epoch+1}/{epochs} [D loss: {d_loss[0]} | D accuracy: {d_loss[1]}] [G loss: {g_loss}]")

train_gan(gan, generator, discriminator, epochs=1000, batch_size=32, train_sequences=train_sequences, latent_dim=latent_dim)

# Generate and plot new wildfire simulations
gan_simulations = generate_simulation(generator, num_samples=5)

for i, sim in enumerate(gan_simulations):
    plt.figure(figsize=(14, 5))
    plt.plot(sim, label=f'Simulation {i+1}')
    plt.legend()
    plt.title('Generated Wildfire Simulation with GAN')
    plt.xlabel('Time Steps')
    plt.ylabel('Normalized Feature Values')
    plt.show()
